## The Challenge

We are an asset manager who has just taken on a new client. The client has given us a number of mandates on how to manage their assets and we have assigned each of these to a different portfolio manager.

Even though there are a number of different mandates, some of the strategies that we are using are common across mandates. We really need to be able to see how each strategy is performing across all of our client's holdings, rather than just looking at a single portfolio in isolation.

## The Solution

Using LUSID we have the ability to add a strategy label to each transaction that we make. We can then aggregate across these labels to get a holistic view into how a strategy is performing. We can do this by:

1. Creating a portfolio for each mandate that our client has given us and group them in a portfolio group
2. Loading our instrument universe 
3. Adding our take on balances for each portfolio which have been provided to us by a transition manager
4. Creating a strategy tag that we can use to tag our transactions with the strategy used to make the trade
5. Adding the first trades that have been made by our portfolio managers inlcuding the strategy that they were following when they made them
6. Creating an analytics store so that we can aggregate across our client's portfolios
7. Aggregate across all of our client's portfolios to see the overall position
8. Aggregate across all of our client's portfolios by strategy to our position per strategy

First things first though, we need to initialise our LUSID environment. 

*To initialise our LUSID environment run the cell below*

In [1]:
# Import LUSID
import lusid.models as models
import lusid_sample_data as import_data

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint

# Authenticate our user and create our API client
client = import_data.authenticate_secrets()

print ('LUSID Environment Initialised')

LUSID Environment Initialised


We now have an empty LUSID environment initialised

![Scopes](img/strategies-lusid.gif)

### 1) Creating a portfolio for each mandate and grouping them in a portfolio group

In LUSID we can create separate environments for each of our clients using scopes. A scope is a container for LUSID objects and can be thought of as a separate identity namespace. Using LUSID's entitlements engine we can ensure that only the right people can access a given scope. Furthermore we can ensure that only the right people can access the right objects inside a given scope. LUSID allows for entitlements down to the most granular level. 

For this example we will create a scope for our new client. Let us assume that our new client is a pension fund for a large high street retailer which we will call 'high_street_pension_fund'.

*Run the cell below to initialise our scope. Note that the scope will have a unique 4 character code appended to it to make the name more unique.*

In [2]:
# Fetch our scopes
scope_id = import_data.create_scope_ids(1)
client_scope_code = 'client-high_street_pension_fund-{}'.format(scope_id[0])
# Pretty print our scope
prettyprint.heading('New Client Scope Code', client_scope_code)

New Client Scope Code: client-high_street_pension_fund-a0fb


We now have a scope for our client inside our LUSID environment. 

![Scopes](img/strategies-scope.gif)

Now that we have our scope we can create our portfolios.

The first thing that we need to do here is give each portfolio a unique name. In LUSID these are refered to as 'codes'. Almost every object in LUSID can be refered to by the combination of the scope that it is contained in and its code. The code must be unique to the scope.

Our pension fund has five mandates and thus five portfolios. 

In LUSID we can use portfolio groups to group the different portfolios of each of our clients together. This allows us to easily keep track of and work with the holdings of each of our clients.

*Run the cell below to import the name for our portfolio group and its corresponding portfolios.*

In [3]:
client_portfolios = import_data.fetch_portfolio_names('strategy-portfolios.csv')
client_group_code = list(client_portfolios.keys())[0]
prettyprint.portfolios(client_portfolios)

Portfolio group: high_street_pension_fund-portfolios
   Portfolio: mandate-balanced
   Portfolio: mandate-energy
   Portfolio: mandate-fixedincome
   Portfolio: mandate-international
   Portfolio: mandate-usgovt




Note that in LUSID we can only create one portfolio at a time. 
        
We will thus create each of the portfolios for our client one after the other and then we will create the portfolio group that binds them together.
        
We on-boarded this clients 5 days ago. When we create the portfolios we can set the creation date using the 'created' argument. This will allow us to backdate holdings and transactions.

You can read more about creating a portfolio in the LUSID API documentation: [LUSID API Docs: Creating a Portfolio](https://docs.lusid.com/#operation/CreatePortfolio)

You can also read more about creating a portfolio group in the LUSID API documentation:

[LUSID API Docs: Creating a Portfolio Group](https://docs.lusid.com/#operation/CreatePortfolioGroup)

*Run the cell below to create our portfolios and group them together*

In [4]:
# Set the creation date of our portfolio
created_date = datetime.now(pytz.UTC) - timedelta(days=5)

# Initialise a list to hold the ids of our portfolios for use in creating our portfolio group
portfolio_resourceids = []

# Loop over our portfolios
for portfolio_code in client_portfolios[client_group_code]:
    # Create the request to add our portfolio
    portfolio_request = models.CreateTransactionPortfolioRequest(display_name=portfolio_code,
                                                                 code=portfolio_code,
                                                                 base_currency='GBP',
                                                                 description=portfolio_code,
                                                                 created=created_date)
    # Call LUSID to create our portfolio
    portfolio_response = client.create_portfolio(scope=client_scope_code,
                                                 create_request=portfolio_request)
    
    # Print the response from LUSID using pretty formatting
    prettyprint.portfolio_response(portfolio_response)

    # Add the portfolio to our list of portfolios for the portfolio group
    portfolio_resourceids.append(
        models.ResourceId(scope=client_scope_code, code=portfolio_code))

# Create our portfolio group request
portfolio_group_request = models.CreatePortfolioGroupRequest(
    id=client_group_code,
    display_name=client_group_code,
    values=portfolio_resourceids,
    description='Grouping all of {}'.format(client_group_code))

# Call LUSID to create our portfolio group
portfolio_group_response = client.create_portfolio_group(scope=client_scope_code,
                                                         request=portfolio_group_request)

# Print the response from LUSID using pretty formatting
prettyprint.portfolio_group_response(portfolio_group_response)

Portfolio Created
Scope: client-high_street_pension_fund-a0fb
Code: mandate-balanced
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:41:49.375131+00:00

Portfolio Created
Scope: client-high_street_pension_fund-a0fb
Code: mandate-energy
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:41:49.746179+00:00

Portfolio Created
Scope: client-high_street_pension_fund-a0fb
Code: mandate-fixedincome
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:41:50.134609+00:00

Portfolio Created
Scope: client-high_street_pension_fund-a0fb
Code: mandate-international
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:41:50.739449+00:00

Portfolio Created
Scope: client-high_street_pension_fund-a0fb
Code: mandate-usgovt
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:41:51.138297+00:

Our scope for our new client is now populated with our portfolios which are grouped together in a portfolio group. All of our portfolios are currently empty.

![Scopes](img/strategies-portfolios.gif)

### 2) Adding our instrument universe

Before we can load our take on balances we first need to add securities to our instrument master in LUSID. 
        
The instrument master contains the details of all of the securities that we hold or trade in. Any time that we conduct a trade or add a holding of a security we refer to it in the instrument master. 

Below we can import our universe of instruments which all of our holdings are made up of. Using LUSID this universe can be pre-populated in advance with your entire instrument universe so that we don't have to keep adding new instruments when we make a trade or update a holding. 

*Run the cell below to import our instrument universe*

In [5]:
instrument_universe = import_data.fetch_instrument_universe('instruments.csv')
prettyprint.instruments(instrument_universe, 'ClientInternal')

Amazon_Nasdaq_AMZN: ClientInternal - US0231351067
Apple_Nasdaq_AAPL: ClientInternal - US0378331005
BP_LondonStockEx_BP: ClientInternal - GB0007980591
BurfordCapital_LondonStockEx_BUR: ClientInternal - GG00B4L84979
EKFDiagnostics_LondonStockEx_EKF: ClientInternal - GB0031509804
Glencore_LondonStockEx_GLEN: ClientInternal - JE00B4T3BW64
JustEat_LondonStockEx_JE: ClientInternal - GB00BKX5CN86
Kingfisher_LondonStockEx_KGF: ClientInternal - GB0033195214
MicroFocus_LondonStockEx_MCRO: ClientInternal - GB00BD8YWM01
RELXGroup_LondonStockEx_REL: ClientInternal - GB00B2B0DG97
Sage_LondonStockEx_SGE: ClientInternal - GB00B8C3BL03
TESCO_LondonStockEx_TSCO: ClientInternal - GB0008847096
UKGiltTreasury_2.0_2025: ClientInternal - GB00BTHH2R79
UKGiltTreasury_3.5_2045: ClientInternal - GB00BN65R313
UKGiltTreasury_3.75_2021: ClientInternal - GB00B4RMG977
UKGiltTreasury_4.5_2034: ClientInternal - GB00B52WS153
USTreasury_2.00_2021: ClientInternal - US912828U816
USTreasury_6.875_2025: ClientInternal - US91

Now that we have our instruments we can add them to LUSID.

To add securities to the instrument master in LUSID we use an upsert method. This method will insert a new record if it does not exist and update the existing record if it does exist. This allows us to add instruments more simply than have to check if an instrument exists before we insert it. 

We can upsert our instruments individually or in a batch. You can read more about upserting instruments
in the LUSID API Documentation: [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments)

*Run the cell below to upsert our instrument universe*

In [6]:
# Initialise our batch upsert request
batch_upsert_request = {}
# Using our instrument universe create our batch request
for instrument_name, instrument in instrument_universe.items():
    batch_upsert_request[instrument_name] = models.InstrumentDefinition(
        name=instrument_name,
        identifiers={'ClientInternal': instrument['identifiers']['ClientInternal']})
    
# Call LUSID to upsert our batch
instrument_upsert_response = client.upsert_instruments(requests=batch_upsert_request)

# Print the response from LUSID using pretty formatting
prettyprint.instrument_response(instrument_upsert_response)

Instrument Successfully Upserted: Amazon_Nasdaq_AMZN
ClientInternal ID: US0231351067
LUSID Instrument ID: LUID_DGD0VFJM


Instrument Successfully Upserted: EKFDiagnostics_LondonStockEx_EKF
ClientInternal ID: GB0031509804
LUSID Instrument ID: LUID_DT3XB8ID


Instrument Successfully Upserted: MicroFocus_LondonStockEx_MCRO
ClientInternal ID: GB00BD8YWM01
LUSID Instrument ID: LUID_TAWHFTHF


Instrument Successfully Upserted: UKGiltTreasury_3.5_2045
ClientInternal ID: GB00BN65R313
LUSID Instrument ID: LUID_0NGQ9Q1U


Instrument Successfully Upserted: Apple_Nasdaq_AAPL
ClientInternal ID: US0378331005
LUSID Instrument ID: LUID_EPIKOEAM


Instrument Successfully Upserted: USTreasury_2.00_2021
ClientInternal ID: US912828U816
LUSID Instrument ID: LUID_C841C1Y6


Instrument Successfully Upserted: BurfordCapital_LondonStockEx_BUR
ClientInternal ID: GG00B4L84979
LUSID Instrument ID: LUID_TUWJRQAD


Instrument Successfully Upserted: USTreasury_6.875_2025
ClientInternal ID: US912810EV62
LUSID Instrum

Our LUSID environment now has an instrument master containing all of our instruments that we will be trading.

![Scopes](img/strategies-instrumentmaster.gif)

During the upsertion process you may have noticed that that every instrument that is created in LUSID given a unique LUSID Instrument Id or LUID for short. This ID is used for many methods and is how LUSID uniquely identifies an instrument.

We therefore want to add our newly created LUIDs to our instrument list so that we can easily reference our instruments in future.

Note that in addition to our instruments, there is also cash in our holdings. Instruments that represent cash are already set-up in LUSID by default. In LUSID the identifier for cash is the prefix the ISO4217 currency code. For example British Pounds have am identifier of GBP. We will add these to our local instrument universe so that we can reference them.

*Run the cell below to add our LUIDs and cash to our local instrument universe.*

In [7]:
# Loop over our recently upserted instruments
for instrument_name, instrument in instrument_upsert_response.values.items():
    # Add our LUID as a new identifier so that we can use it in our calls later
    instrument_universe[instrument_name]['identifiers']['LUID'] = instrument.lusid_instrument_id
    
# We will also add some cash instruments to our universe
instrument_universe['GBP_Cash'] = {
    'identifiers': {'LUID': 'GBP'},
    'currency': 'GBP'}

instrument_universe['USD_Cash'] = {
    'identifiers': {'LUID': 'USD'},
    'currency': 'USD'}
            
prettyprint.instruments(instrument_universe, 'LUID')

Amazon_Nasdaq_AMZN: LUID - LUID_DGD0VFJM
Apple_Nasdaq_AAPL: LUID - LUID_EPIKOEAM
BP_LondonStockEx_BP: LUID - LUID_6DHAL5AS
BurfordCapital_LondonStockEx_BUR: LUID - LUID_TUWJRQAD
EKFDiagnostics_LondonStockEx_EKF: LUID - LUID_DT3XB8ID
Glencore_LondonStockEx_GLEN: LUID - LUID_MFN7AIC4
JustEat_LondonStockEx_JE: LUID - LUID_AZDMPJDT
Kingfisher_LondonStockEx_KGF: LUID - LUID_C70AW5JB
MicroFocus_LondonStockEx_MCRO: LUID - LUID_TAWHFTHF
RELXGroup_LondonStockEx_REL: LUID - LUID_AQQL94N5
Sage_LondonStockEx_SGE: LUID - LUID_07ONE9PV
TESCO_LondonStockEx_TSCO: LUID - LUID_348HKRHV
UKGiltTreasury_2.0_2025: LUID - LUID_8SA5LVGX
UKGiltTreasury_3.5_2045: LUID - LUID_0NGQ9Q1U
UKGiltTreasury_3.75_2021: LUID - LUID_KEQGUCLQ
UKGiltTreasury_4.5_2034: LUID - LUID_4C63M0G5
USTreasury_2.00_2021: LUID - LUID_C841C1Y6
USTreasury_6.875_2025: LUID - LUID_IR6AC9IV
WPP_LondonStockEx_WPP: LUID - LUID_B4W31L09
Whitebread_LondonStockEx_WTB: LUID - LUID_64KMOWWV
GBP_Cash: LUID - GBP
USD_Cash: LUID - USD


Now that we have added the LUSID Instrument Id to our local instrument universe we are ready to set our take on balances.

### 3) Adding our take on balances for each portfolio which have been provided to us by a transition manager

We have been advised by a transition manager on the holdings that we will be taking on for our new client. They have provided these initial holdings to us in a CSV.

Let us load our take on balances. 

*Run the cell below to import our initial holdings information*

In [8]:
client_holdings = import_data.fetch_client_take_on_balances('strategy-holdings.csv')
prettyprint.holdings(client_holdings)

Portfolio: mandate-balanced

Instrument Name: Kingfisher_LondonStockEx_KGF
Quantity: 1362038
CostPrice: 2.276

Instrument Name: JustEat_LondonStockEx_JE
Quantity: 834553
CostPrice: 5.4639999999999995

Instrument Name: RELXGroup_LondonStockEx_REL
Quantity: 494343
CostPrice: 15.98

Instrument Name: UKGiltTreasury_4.5_2034
Quantity: 77481
CostPrice: 140.572

Instrument Name: GBP_Cash
Quantity: 952000
CostPrice: 1.0




Portfolio: mandate-energy

Instrument Name: Glencore_LondonStockEx_GLEN
Quantity: 905141
CostPrice: 2.762

Instrument Name: BP_LondonStockEx_BP
Quantity: 1713922
CostPrice: 5.114

Instrument Name: GBP_Cash
Quantity: 2200000
CostPrice: 1.0




Portfolio: mandate-fixedincome

Instrument Name: UKGiltTreasury_3.5_2045
Quantity: 266169
CostPrice: 134.433

Instrument Name: UKGiltTreasury_2.0_2025
Quantity: 405589
CostPrice: 106.637

Instrument Name: UKGiltTreasury_3.75_2021
Quantity: 174800
CostPrice: 108.126

Instrument Name: USTreasury_2.00_2021
Quantity: 357507
CostPrice: 97.9

To set our holdings in LUSID we use the set holdings method. This method takes a list of adjustment requests. Using these requests LUSID will determine what transactions are required to update our portfolios so that they have the desired holdings.

We will make our holdings effecitve as of five days ago as this was when we on-boarded the client. 

Just like when we created our portfolios we can only set the holdings one portfolio at a time. 

You can read more about setting the holdings on a portfolio in the LUSID API Documentation: [LUSID API Docs: Setting the Holdings on a Portfolio](https://docs.lusid.com/#operation/SetHoldings)

*Run the cell below to set our take on balances*

In [9]:
holdings_effective_date = (datetime.now(pytz.UTC) - timedelta(days=5))

# Iterate over our portfolios
for portfolio_name, portfolio in client_holdings.items():
    
    # Initialise our list to hold the holding adjustments
    holding_adjustments = []

    # Iterate over the holdings in each portfolio
    for instrument_name, holding in portfolio.items():
        if 'Cash' in instrument_name:
            identifier_key = 'Instrument/default/Currency'
        else:
            identifier_key = 'Instrument/default/LusidInstrumentId'
            
        Luid = instrument_universe[instrument_name]['identifiers']['LUID']

        # Create our adjust holdings request using our instrument universe to get the LUID identifier for the instrument
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={
                    identifier_key: Luid},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding['quantity'],
                        cost=models.CurrencyAndAmount(
                            amount=holding['quantity'] * holding['price'],
                            currency=instrument_universe[instrument_name]['currency']),
                        portfolio_cost=holding['quantity'] * holding['price'],
                        price=holding['price'])
                ])
        )
    
    # Call LUSID to set our holdings
    set_holdings_response = client.set_holdings(scope=client_scope_code,
                                                code=portfolio_name,
                                                effective_at=holdings_effective_date,
                                                holding_adjustments=holding_adjustments)
    
    # Print the response from LUSID using pretty formatting
    prettyprint.set_holdings_response(set_holdings_response, 
                                      client_scope_code, 
                                      portfolio_name)

print ('Holdings have been set for all Portfolios')

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-balanced
Holdings Effective From: 2019-02-07 21:41:52.260439+00:00
Holdings Created On: 2019-02-12 21:41:52.541895+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-energy
Holdings Effective From: 2019-02-07 21:41:52.260439+00:00
Holdings Created On: 2019-02-12 21:41:52.979407+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-fixedincome
Holdings Effective From: 2019-02-07 21:41:52.260439+00:00
Holdings Created On: 2019-02-12 21:41:53.393286+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-international
Holdings Effective From: 2019-02-07 21:41:52.260439+00:00
Holdings Created On: 2019-02-12 21:41:53.855412+00:00

Holdings Successfully Set for Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-usgovt
Holdings 

Our portfolios are no longer empty and now contain the initial holdings for our new client.

![Scopes](img/strategies-holdings.gif)

### 4) Creating a strategy tag that we can use to tag our transactions with the strategy used to make the trade

We want to be able to track what strategies our portfolio managers were following when they made an investment decision. To do this in LUSID we can use properties. We can define a property and attach it to almost any object in LUSID. 

In this case we can create a 'strategy' property that we can attach to our transactions. This property will take the form of a string which will be the name of the strategy that the portfolio manager followed. Examples of strategies in this case are 'quantitativeSignal', 'fundamentalAnalysis' etc.

You can read more about defining new properties in the LUSID API documentation: [LUSID API Docs: Creating a New Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition)

*Run the cell below to create our strategy property*

In [10]:
# Create our request to define a new property
property_request = models.CreatePropertyDefinitionRequest(
    domain='Trade',
    scope=client_scope_code,
    code='strategy',
    value_required=False,
    display_name='strategy',
    data_type_id=models.ResourceId(scope='default', code='string'))

# Call LUSID to create our new property
property_response = client.create_property_definition(definition=property_request)

# Grab the key off the response to use when referencing this property in other LUSID calls
strategy_property_key = property_response.key

# Pretty print our key
prettyprint.heading('Strategy Property Key', strategy_property_key)

Strategy Property Key: Trade/client-high_street_pension_fund-a0fb/strategy


We now have a strategy property that we can set on each of our transactions which is referenced by our strategy property key.

![Scopes](img/strategies-strategypropertykey.gif)

### 5) Adding the first trades that have been made by our portfolio managers inlcuding the strategy that they were following when they made them

Now that we have our transaction property defined we can load our transactions. We will load these from a CSV file. In practice these are likely to come from our order management system. You can see from looking at the transaction dates that these transactions have all occured over the last 4 days.

*Run the cell below to import our transactions*

In [11]:
client_transactions = import_data.fetch_client_transactions('strategy-transactions.csv', 
                                                            instrument_universe,
                                                            days_back=4,
                                                            portfolios=True)

prettyprint.transactions_strategy(client_transactions, portfolios=True)

Portfolio: mandate-balanced

Transaction: tid_329432525234324
Instrument Name: Kingfisher_LondonStockEx_KGF
Instrument Id: LUID_C70AW5JB
Type: Sell
Units: 325000
Price: 2.345
Total Cost: 762125.0
Currency: GBP
Transaction Date: 2019-02-09T09:52:54.165879+00:00
Settlement Date: 2019-02-11T09:52:54.165879+00:00
Description: Equity Sale
Strategy: quantitativeSignal

Transaction: tid_325452342424500
Instrument Name: UKGiltTreasury_4.5_2034
Instrument Id: LUID_4C63M0G5
Type: Buy
Units: 10501
Price: 140.572
Total Cost: 1476146.572
Currency: GBP
Transaction Date: 2019-02-09T10:39:26.836678+00:00
Settlement Date: 2019-02-11T10:39:26.836678+00:00
Description: Equity Purchase
Strategy: incomeRequirements



Portfolio: mandate-fixedincome

Transaction: tid_234295929052090
Instrument Name: UKGiltTreasury_3.75_2021
Instrument Id: LUID_KEQGUCLQ
Type: Buy
Units: 24000
Price: 109.126
Total Cost: 2619024.0
Currency: GBP
Transaction Date: 2019-02-09T13:30:15.070268+00:00
Settlement Date: 2019-02-11T13:3

Now that we have our transactions we can upsert them into LUSID. We can only upsert transactions for one portfolio at time. Therefore we will upsert our transactions in batches. Each batch contains all the transactions for a given portfolio. 

You can read more about upserting transactions in the LUSID API documentation: [LUSID API Docs: Upserting Transactions](https://docs.lusid.com/#operation/UpsertTransactions)

*Run the cell below to upsert our transactions*

In [12]:
# Iterate over our portfolios
for portfolio_name, transactions in client_transactions.items():
    # Initialise a list to hold our transactions for each portfolio
    batch_transaction_requests = []
    # Iterate over the transactions for each portfolio
    for transaction_id, transaction in transactions.items():
        # Append the transaction to our request, note the use of the 'properties' argument
        batch_transaction_requests.append(
            models.TransactionRequest(
                transaction_id=transaction_id,
                type=transaction['type'],
                instrument_identifiers={
                    'Instrument/default/LusidInstrumentId': transaction['instrument_uid']},
                transaction_date=transaction['transaction_date'],
                settlement_date=transaction['settlement_date'],
                units=transaction['units'],
                transaction_price=models.TransactionPrice(
                  price=transaction['transaction_price'],
                  type='Price'),
                total_consideration=models.CurrencyAndAmount(
                  amount=transaction['total_cost'],
                  currency=transaction['transaction_currency']),
                 source='Client',
                 transaction_currency=transaction['transaction_currency'],
                 properties={
                     strategy_property_key: models.PropertyValue(
                         label_value=transaction['strategy'])}
              ))
            
    # Call LUSID to upsert our transactions
    transaction_response = client.upsert_transactions(
        scope=client_scope_code,
        code=portfolio_name,
        transactions=batch_transaction_requests)
    
    # Print the response from LUSID using pretty formatting 
    prettyprint.transactions_response(transaction_response,
                                      client_scope_code,
                                      portfolio_name)

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-balanced
Transactions Effective From: 2019-02-09 10:39:26.836678+00:00
Transactions Created On: 2019-02-12 21:41:55.388812+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-fixedincome
Transactions Effective From: 2019-02-10 13:19:57.270710+00:00
Transactions Created On: 2019-02-12 21:41:55.920885+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-energy
Transactions Effective From: 2019-02-11 15:53:06.067211+00:00
Transactions Created On: 2019-02-12 21:41:56.431786+00:00

Transactions Successfully Upserted into Portfolio
Scope: client-high_street_pension_fund-a0fb
Code: mandate-usgovt
Transactions Effective From: 2019-02-11 12:35:16.704671+00:00
Transactions Created On: 2019-02-12 21:41:56.943370+00:00

Transactions Successfully Upserted into Portfolio

Now each of our portfolios contains a list of transactions, the first are 'StockIn' transactions generated by LUSID to set our holdings, the second set are the transactions from our portfolio managers which are tagged with a strategy.

![Scopes](img/strategies-transactions.gif)

### 6) Creating an analytics store so that we can aggregate across our client's portfolios

Now that we have populated our portfolios we want to aggregate across them to see both our client's overall position as well as our exposure as the result of each strategy that our portfolio managers are using. 

To this we first need to create an analytics store. This holds the result of our aggregation queries as well as the prices of our instruments.

You can read more about creating an analytics store in the LUSID API documentation: [LUSID API Docs: Creating an Analytics Store](https://docs.lusid.com/#operation/CreateAnalyticStore)

*Run the cell below to create our analytics store*

In [13]:
# Set our analytics effective date to be from now
analytics_effective_date = datetime.now(pytz.UTC)

# Create analytics store request
analytics_store_request = models.CreateAnalyticStoreRequest(
    scope=client_scope_code,
    date_property=analytics_effective_date)

# Call LUSID to create our analytics store
analytic_store = client.create_analytic_store(request=analytics_store_request)

# Pretty print the response from LUSID
prettyprint.analytic_store(analytic_store)

Analytic Store Created
Scope: client-high_street_pension_fund-a0fb
Effective From: 2019-02-12T00:00:00+00:00


Now that we have created our analytic store we need to populate it with prices. As we are not going to be valuing our portfolios at this time we will give each instrument a dummy price of 1. 

You can read more about setting analytics in the LUSID API documentation: [LUSID API Docs: Setting Analytics](https://docs.lusid.com/#operation/SetAnalytics)

*Run the cell below to set our instrument analytics*

In [14]:
# Create prices via instrument, analytic
instrument_analytics = []

# Create dummy prices of 1 for all instruments except cash which we will ignore
for instrument_name, instrument in instrument_universe.items():
    if 'Cash' in instrument_name:
        continue
    instrument_analytics.append(
        models.InstrumentAnalytic(
            instrument_uid=instrument['identifiers']['LUID'],
            value=1))

# Call LUSID to set up our newly created analytics store
analytics = client.set_analytics(scope=client_scope_code,
                                 year=analytics_effective_date.year,
                                 month=analytics_effective_date.month,
                                 day=analytics_effective_date.day,
                                 data=instrument_analytics)

# Pretty print the response from LUSID
prettyprint.analytic_store(analytics, True)

Analytics Set
Scope: client-high_street_pension_fund-a0fb
Effective From: 2019-02-12T00:00:00+00:00


We now have an analytics store in our new client's scope that we can use for our aggregation requests.

![Scopes](img/strategies-analyticsstore.gif)

### 7) Aggregate across all of our client's portfolios to see the overall position

The first thing we want to do is aggregate across our client's entire group of portfolios to see their overall poistion. We can do this by aggregating the holdings by the portfolio group. We will sum the units and the cost of each instrument across the group.

You can read more about creating by portfolio group in the LUSID API documentation: [LUSID API Docs: Aggregate by Portfolio Group](https://docs.lusid.com/#operation/GetAggregationByGroup)

*Run the cell below to aggregate across all of our client's portfolios*

In [15]:
# Create our aggregation request
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=client_scope_code,
        code='default'),
        effective_at=datetime.now(pytz.UTC),
        metrics=[
            models.AggregateSpec(key='Holding/default/SubHoldingKey',
            op='Value'),
            models.AggregateSpec(key='Instrument/default/Name',
            op='Value'),
            models.AggregateSpec(key='Holding/default/Units',
            op='sum'),
            models.AggregateSpec(key='Holding/default/Cost',
            op='sum')
        ],
        group_by=[
            'Holding/default/SubHoldingKey'
        ])

# Call LUSID to aggregate across all of our portfolios
aggregated_group = client.get_aggregation_by_group(
    scope=client_scope_code,
    code=client_group_code,
    request=aggregation_request)

# Pretty print our response from LUSID
prettyprint.aggregation_response(aggregated_group)

LUSID Instrument Id: LUID_C70AW5JB
Instrument Name: Kingfisher_LondonStockEx_KGF
Instrument Units: 1037038.0
Instrument Cost: 2360298.49
Instrument Currency: GBP

LUSID Instrument Id: LUID_AZDMPJDT
Instrument Name: JustEat_LondonStockEx_JE
Instrument Units: 834553.0
Instrument Cost: 4559997.59
Instrument Currency: GBP

LUSID Instrument Id: LUID_AQQL94N5
Instrument Name: RELXGroup_LondonStockEx_REL
Instrument Units: 494343.0
Instrument Cost: 7899601.14
Instrument Currency: GBP

LUSID Instrument Id: LUID_4C63M0G5
Instrument Name: UKGiltTreasury_4.5_2034
Instrument Units: 87982.0
Instrument Cost: 12367805.7
Instrument Currency: GBP

LUSID Instrument Id: CCY_GBP
Instrument Name: Cash
Instrument Units: 789254.43
Instrument Cost: 789254.43
Instrument Currency: GBP

LUSID Instrument Id: LUID_MFN7AIC4
Instrument Name: Glencore_LondonStockEx_GLEN
Instrument Units: 785141.0
Instrument Cost: 2168559.44
Instrument Currency: GBP

LUSID Instrument Id: LUID_6DHAL5AS
Instrument Name: BP_LondonStockEx_

Now we have a view of our client's holdings across their entire group of portfolios.

![Scopes](img/strategies-aggregatedportfoliogroup.gif)

### 8) Aggregate across all of our client's portfolios by strategy to our position per strategy

Now that we have a view of our client's overall position we want to get a sense as to what strategies our portfolio manager's have been using to make their trades and how our position looks by strategy.

To do this we will make us of Derived Portfolios. 

A derived portfolio is a portfolio that is derived from a parent portfolio. It inherits all of its transactions from the parent portfolio. We can make use of derived portfolios to apply sub-holding keys to our holdings. In this case we will use our transaction strategy property as our sub-holding key.

We will create a new scope to hold our derived portfolios. This will allow us to give them the same name as the parent portfolio.

*Run the cell below to create our new scope*

In [16]:
# Fetch our scopes
scope_id = import_data.create_scope_ids(1)
client_strategy_scope_code = 'client-strategy-high_street_pension_fund-{}'.format(scope_id[0])
# Pretty print our scope
prettyprint.heading('New Client Strategy Scope Code', client_scope_code)

New Client Strategy Scope Code: client-high_street_pension_fund-a0fb


Now that we have our new scope we can create our derived portfolios. We will create one for each of the corresponding parent portfolios. We will also group our derived portfolios together so that we can aggregate across the group.

Note that we use the same creation date for the derived portfolios as we did for the parent portfolio. 

You can read more about creating a derived portfolio in the LUSID API documentation: [LUSID API Docs: Creating a Derived Portfolio](https://docs.lusid.com/#operation/CreateDerivedPortfolio)

*Run the cell below to create and group our derived portfolios*

In [17]:
# Initialise a list to hold the ids of our portfolios for use in creating our portfolio group
portfolio_resourceids = []

# Loop over our portfolios
for portfolio_code in client_portfolios[client_group_code]:
    # Create the request to add our portfolio
    portfolio_request = models.CreateDerivedTransactionPortfolioRequest(
        display_name=portfolio_code,
        code=portfolio_code,
        parent_portfolio_id=models.ResourceId(
            scope=client_scope_code,
            code=portfolio_code),
        description=portfolio_code,
        created=created_date,
        sub_holding_keys=[strategy_property_key])
    
    # Call LUSID to create our portfolio
    portfolio_response = client.create_derived_portfolio(
        scope=client_strategy_scope_code,
        portfolio=portfolio_request)
    
    # Print the response from LUSID using pretty formatting
    prettyprint.portfolio_response(portfolio_response)

    # Add the portfolio to our list of portfolios for the portfolio group
    portfolio_resourceids.append(
        models.ResourceId(
            scope=client_strategy_scope_code, 
            code=portfolio_code))

# Create our portfolio group request
portfolio_group_request = models.CreatePortfolioGroupRequest(
    id=client_group_code,
    display_name=client_group_code,
    values=portfolio_resourceids,
    description='Grouping all of {}'.format(client_group_code))

# Call LUSID to create our portfolio group
portfolio_group_response = client.create_portfolio_group(
    scope=client_strategy_scope_code,
    request=portfolio_group_request)

# Print the response from LUSID using pretty formatting
prettyprint.portfolio_group_response(portfolio_group_response)

Portfolio Created
Scope: client-strategy-high_street_pension_fund-f7de
Code: mandate-balanced
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:41:59.177876+00:00

Portfolio Created
Scope: client-strategy-high_street_pension_fund-f7de
Code: mandate-energy
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:41:59.650413+00:00

Portfolio Created
Scope: client-strategy-high_street_pension_fund-f7de
Code: mandate-fixedincome
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:42:00.067490+00:00

Portfolio Created
Scope: client-strategy-high_street_pension_fund-f7de
Code: mandate-international
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfolio Created On: 2019-02-12 21:42:00.499485+00:00

Portfolio Created
Scope: client-strategy-high_street_pension_fund-f7de
Code: mandate-usgovt
Portfolio Effective From: 2019-02-07 21:41:49.014406+00:00
Portfol

![Scopes](img/strategies-derived.gif)

Now that we have created our derived portfolios we can aggregate across our group of derived portfolios. Note that we can use the analytic store from the previous aggregation in our main scope as we only had dummy prices in there.

*Run the cell below to aggregate by strategy across our client's holdings*

In [18]:
aggregation_request = models.AggregationRequest(
    recipe_id=models.ResourceId(
        scope=client_scope_code,
        code='default'),
    effective_at=datetime.now(pytz.UTC),
    metrics=[
        models.AggregateSpec(
            key=strategy_property_key,
            op='value'),
        models.AggregateSpec(
            key='Holding/default/Cost',
            op='sum')],
    group_by=[
        strategy_property_key],
    filters=[
        models.PropertyFilter(
            left=strategy_property_key,
            operator='NotEquals',
            right='<Not Classified>',
            right_operand_type='Absolute'),
        models.PropertyFilter(
            left='Instrument/default/Name',
            operator='NotEquals',
            right='<Unknown>',
            right_operand_type='Absolute')
    ]
    )

aggregated_group = client.get_aggregation_by_group(scope=client_strategy_scope_code,
                                                   code=client_group_code,
                                                   request=aggregation_request)

In [19]:
# Pretty print our response from LUSID
prettyprint.aggregation_response_strategy(aggregated_group, strategy_property_key)

Strategy: quantitativeSignal
Strategy Cost: -762125.0

Strategy: incomeRequirements
Strategy Cost: 1348370.57

Strategy: fundamentalAnalysis
Strategy Cost: -47650.0

Strategy: internationalExposure
Strategy Cost: -5574600.0

Strategy: newsSentiment
Strategy Cost: 2527350.0



We can now see how much we have invested into each of the different investment strategies. Our final state looks like the below.

![Scopes](img/strategies-strategyaggregation.gif)